In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import base64

from weasyprint import HTML

In [ ]:
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

In [ ]:
# Google's API Python Fast Start


def main():
  """Shows basic usage of the Gmail API.
  Lists the user's Gmail labels.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)
    results = service.users().labels().list(userId="me").execute()
    labels = results.get("labels", [])

    if not labels:
      print("No labels found.")
      return
    print("Labels:")
    for label in labels:
      print(label["name"])

  except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

In [ ]:
"""
Loads credentials and builds a service object using the respective endpoint.
Then, it executes the query to retrieve the messages.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
  creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
  if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())
  else:
    flow = InstalledAppFlow.from_client_secrets_file(
        "credentials.json", SCOPES
    )
    creds = flow.run_local_server(port=0)
  # Save the credentials for the next run
  with open("token.json", "w") as token:
    token.write(creds.to_json())
try:
  # Call the Gmail API
  service = build("gmail", "v1", credentials=creds)
  results = service.users().messages().list(userId="me",q="from:journiehelp@artisoftlabs.com is:unread").execute()
except HttpError as error:
  # TODO(developer) - Handle errors from gmail API.
  print(f"An error occurred: {error}")



print("Results: ")
results

In [ ]:
# Get the HTML payload data for each item in the results list.

resulting_htmls = []

for item in results['messages']:

    item_id = item['id']
    email = service.users().messages().get(userId="me",id=item_id,format="full").execute()

    html_body = base64.urlsafe_b64decode(email["payload"]["parts"][1]["body"]["data"]).decode('utf-8') 

    resulting_htmls.append(html_body)


resulting_htmls

In [ ]:
# Convert HTMLs to PDF file.

counter = 0
for html_item in resulting_htmls:
    
    toWriteItem = HTML(string=html_item)
    
    outFile = f"out_{counter}.pdf"

    toWriteItem.write_pdf(target=outFile)

    counter+=1



# Command terminal prompt
weasyprint test2.html out.pdf